![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/min-max-scaler.png)

# Min-Max Scaler


In [ ]:
import azureml.dataprep as dprep

The min-max scaler scales all values in a column to a desired range (typically [0, 1]). This is also known as feature scaling or unity-based normalization. Min-max scaling is commonly used to normalize numeric columns in a data set for machine learning algorithms.

First, load a data set containing information about crime in Chicago. Keep only a few columns.

In [ ]:
dflow = dprep.read_csv('../data/crime-spring.csv')
dflow = dflow.keep_columns(columns=['ID', 'District', 'FBI Code'])
dflow = dflow.to_number(columns=['District', 'FBI Code'])
dflow.head(5)

Using `get_profile()`, you can see the shape of the numeric columns such as the minimum, maximum, count, and number of error values.

In [ ]:
dflow.get_profile()

To apply min-max scaling, call the function `min_max_scaler` on the Dataflow and specify the column name. This will trigger a full data scan over the column to determine the min and max values and perform the scaling. Note that the min and max values of the column are preserved at this point. If the same dataflow steps are performed over a different dataset, the min-max scaler must be re-executed.

In [ ]:
dflow_district = dflow.min_max_scale(column='District')
dflow_district.head(5)

Look at the data profile to see that the "District" column is now scaled; the min is 0 and the max is 1. Any error values and missing values from the source column are preserved.

In [ ]:
dflow_district.get_profile()

You can also specify a custom range for the scaling. Instead of [0, 1], let's choose [-10, 10].

In [ ]:
dflow_district_range = dflow.min_max_scale(column='District', range_min=-10, range_max=10)
dflow_district_range.head(5)

In some cases, you may want to manually provide the min and max of the data in the source column. For example, you may want to avoid a full data scan because the dataset is large and we already know the min and max. You can provide the known min and max to the `min_max_scaler` function. The column will be scaled using the provided values. For example, if you want to scale the `FBI Code` column with 6  (`data_min`) becoming 0 (`range_min`), the program will scan the data to get `data_max`, which will become 1 (`range_max`).

In [ ]:
dflow_fbi = dflow.min_max_scale(column='FBI Code', data_min=6)
dflow_fbi.get_profile()

## Using a Min-Max Scaler builder

For more flexibility when constructing the arguments for the min-max scaling, you can use a Min-Max Scaler builder.

In [ ]:
builder = dflow.builders.min_max_scale(column='District')
builder

Calling `builder.learn()` will trigger a full data scan to see what `data_min` and `data_max` are. You can choose whether to use these values or set custom values.

In [ ]:
builder.learn()
builder

If you want to provide custom values for any of the arguments, you can update the builder object.

In [ ]:
builder.range_max = 10
builder.data_min = 6
builder

When you are satisfied with the arguments, you will call `builder.to_dataflow()` to get the result. Note that the min and max values of the source column is preserved by the builder at this point. If you need to get the true `data_min` and `data_max` values again, you will need to set those arguments on the builder to `None` and then call `builder.learn()` again.

In [ ]:
dflow_builder = builder.to_dataflow()
dflow_builder.head(5)